## Dragon real state house :- price predictor

In [1]:
import pandas as pd
import numpy as np

In [2]:
housing = pd.read_csv("data.csv")

In [3]:
# housing.head()
# housing.info()
# housing["CHAS"].value_counts()
# housing.describe()

In [4]:

# %matplotlib inline
# import matplotlib.pyplot as plt
# housing.hist(bins=50, figsize=(25, 20))
# plt.show()

## Train test splitting for learning purpose

In [5]:
# import numpy as np
# def split_train_test(data, test_ratio):
#     np.random.seed(42)    #seed is fix the shuffle
#     shuffled = np.random.permutation(len(data))
#     test_set_size = int(len(data) * test_ratio)
#     test_indices = shuffled[:test_set_size]
#     train_indices = shuffled[test_set_size:]    
#     return data.iloc[train_indices], data.iloc[test_indices]
# train_set , test_set = split_train_test(housing, 0.2)
# print(f"Rows in train set{train_set}\nRows in test set: {test_set}\n")

## Train test through SKlearn

In [6]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
print(f"Rows in train set{len(train_set)}\nRows in test set: {len(test_set)}\n")

Rows in train set404
Rows in test set: 102



In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
# strat_test_set['CHAS'].value_counts()

In [8]:
housing = strat_train_set.copy() # Before we start fillin missing attribute values

## Looking for correlation

In [9]:
# corr_matrix = housing.corr()
# corr_matrix['MEDV'].sort_values(ascending=False)

In [10]:
# from pandas.plotting import scatter_matrix
# attributes = ["MEDV","RM","ZN","LSTAT"]
# scatter_matrix(housing[attributes], figsize = (12,8))

In [11]:
# housing.plot(kind = "scatter",x="RM",y="MEDV",alpha = 0.8) 

## Trying out Attribute combinations

In [12]:
# housing['TAXRM'] = housing["TAX"]/housing["RM"]  # making a new attribute
# housing.head()

In [13]:
# corr_matrix = housing.corr()
# corr_matrix['MEDV'].sort_values(ascending=False)

## Seperate labels and Features

In [14]:
housing = strat_train_set.drop("MEDV",axis=1)
housing_labels = strat_train_set["MEDV"].copy()

## Missing attribute

    To take care of missing attributes, you have three options :
        1. Get rid of the missing data points
        2. Get rid of the whole attribute
        3. Set the value to some value(0, mean or median)

In [15]:
# a = housing.dropna(subset=["RM"]) # Option 1
# a.shape                           # Note :- Original dataframe is not change

In [16]:
# housing.drop("RM", axis = 1).shape # Option 2 means RM column is deleted  # Note :- Original dataframe is not change

In [17]:
# median = housing["RM"].median()  #Option 3
# housing["RM"].fillna(median)    # Note :- Original dataframe is not change

In [18]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = "median") # Its use option 3rd 
imputer.fit(housing)

SimpleImputer(strategy='median')

In [19]:
# imputer.statistics_.shape

In [20]:
X = imputer.transform(housing)
housing_tr = pd.DataFrame(X, columns = housing.columns)
housing_tr.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.602814,10.836634,11.344950,0.069307,0.558064,6.280559,69.039851,3.746210,9.735149,412.341584,18.473267,353.392822,12.791609
std,8.099383,22.150636,6.877817,0.254290,0.116875,0.712633,28.258248,2.099057,8.731259,168.672623,2.129243,96.069235,7.235740
min,0.006320,0.000000,0.740000,0.000000,0.389000,3.561000,2.900000,1.129600,1.000000,187.000000,13.000000,0.320000,1.730000
25%,0.086962,0.000000,5.190000,0.000000,0.453000,5.878750,44.850000,2.035975,4.000000,284.000000,17.400000,374.617500,6.847500
50%,0.286735,0.000000,9.900000,0.000000,0.538000,6.211000,78.200000,3.122200,5.000000,337.000000,19.000000,390.955000,11.570000
75%,3.731923,12.500000,18.100000,0.000000,0.631000,6.630250,94.100000,5.100400,24.000000,666.000000,20.200000,395.630000,17.102500
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


## Scikit-learn Design

    Primarily, three types of objects
    
        1. Estimators - It estimates some parameter Ex:= Imputer
            * It has a fit method and transform method
            * Fit method - Fits the dataset and calculates internal parameters
            
        2. Transformers - Transform method takes input and returns output based on the learnings from fit().It also has a 
           convienience fuction called fit_transform() which fit then transform.
           
        3. Predictors - LinearRegression model is an exp of predictor.It has fit() and predict() are two common functions. 
           It also gives score() function which will evaluate the predictions.

## Feature Scaling
    
    Primarilly, two types of feature scaling methods:
    1. Min-max scaling (Normalization):-
        (value-min)/(max-min)
        Sklearn provide a class called MinMaxscaler for this 
    2. Standardization:-
        (value - mean)/(standard deviation)
        SKlearn provides a class called StandardScaler for this

## Crreating a Pipeline

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = "median")), 
    # ... add as many as you want in your pipeline
    ('std_scaler', StandardScaler())
])


In [22]:
housing_num_tr = my_pipeline.fit_transform(housing)
print(housing_num_tr.shape)

(404, 13)


## Selecting a desired model for Rk Real Estates

In [23]:
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# model = LinearRegression()      # Model 1 Good
# model = DecisionTreeRegressor() # Model 2 Better then 1
model = RandomForestRegressor()   # Model 3 Best for this
model.fit(housing_num_tr, housing_labels)

RandomForestRegressor()

In [24]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
prepared_data = my_pipeline.transform(some_data)

In [25]:
model.predict(prepared_data)

array([22.382, 25.702, 16.534, 23.267, 23.524])

In [26]:
list(some_labels)

[21.9, 24.5, 16.7, 23.1, 23.0]

## Evaluating the model

In [27]:
from sklearn.metrics import mean_squared_error
housing_predictions = model.predict(housing_num_tr)
mse = mean_squared_error(housing_labels, housing_predictions)
rmse = np.sqrt(mse)

In [28]:
rmse # show 0.0 error means over fitting is here
    # Lets solve it in next cell

1.1757541639136961

## Using better evalution technique - Cross Validation

In [29]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, housing_num_tr, housing_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

In [30]:
rmse_scores

array([2.92569595, 2.80332097, 4.47994276, 2.58478323, 3.48216136,
       2.597181  , 4.76065867, 3.29038004, 3.41817573, 3.26251319])

In [31]:
def print_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())    
    print("Standard deviation:",scores.std())    

In [32]:
print_scores(rmse_scores)

Scores: [2.92569595 2.80332097 4.47994276 2.58478323 3.48216136 2.597181
 4.76065867 3.29038004 3.41817573 3.26251319]
Mean: 3.36048129010378
Standard deviation: 0.7021494175267581


## Saving The Model

In [34]:
from joblib import dump, load
dump(model, "RkRealEstate.joblib")

['RkRealEstate.joblib']

## Testing the model on test data

In [35]:
X_test = strat_test_set.drop("MEDV", axis = 1)
Y_test = strat_test_set["MEDV"].copy()
X_test_prepared = my_pipeline.transform(X_test)
final_predictions = model.predict(X_test_prepared)
final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [36]:
print(final_rmse)

2.9262916789336075


In [38]:
print(final_predictions,"\n", list(Y_test))

[24.244 11.446 25.598 21.761 18.043 14.81  20.616 14.467 32.548 41.714
 19.406 11.348 25.568 28.4   19.261 11.093 31.912 14.426 23.543 18.935
 20.126 17.882 17.036 21.85  18.26  31.583 15.958 33.122  8.935 33.916
 23.984 21.401 22.861 10.814 20.936 10.993 42.85  24.85  23.25  41.052
 23.892 30.76  20.748 20.836 18.984 33.344 44.651 20.175 20.476 21.662
 21.046 14.18  21.652 14.977 25.181 33.456 42.147 29.449 19.368 20.796
 46.692  9.471 18.825 25.337 14.379 33.099 20.146 17.957 19.119 34.367
 25.682 22.761 21.689 22.725 34.392 12.657 15.52  19.713 21.08  21.276
 22.712 21.355 13.747 22.827 20.686 21.311 13.946 21.055 22.131 23.063
 18.463 26.477  6.682 26.672 19.008 29.43  19.546 31.237 14.33  26.52
 21.226 20.2  ] 
 [16.5, 10.2, 30.1, 23.0, 14.4, 15.6, 19.4, 14.1, 30.3, 35.2, 23.1, 13.8, 25.0, 27.9, 19.5, 12.3, 32.2, 13.5, 23.8, 21.7, 19.2, 19.5, 10.4, 23.2, 18.6, 28.5, 15.2, 32.0, 7.2, 34.6, 20.1, 20.6, 23.6, 13.1, 23.8, 12.7, 43.1, 24.7, 22.2, 44.0, 28.1, 31.0, 21.7, 23.4, 19.5, 33.